Maxwell solver with mixed BC
============================
**keys**: mixed bvp, MoM, PEC scattering

In [ ]:
import sys
sys.path.append("../build/")
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG, GMRes

In [ ]:
topsphere = Sphere((0,0,0), 1) * Box((-1,-1,0),(1,1,1))
botsphere = Sphere((0,0,0), 1) - Box((-1,-1,0),(1,1,1))
topsphere.faces.name = "neumann" # 1/kappa n x curl E
botsphere.faces.name = "dirichlet" # nxE
shape = Fuse( [topsphere,botsphere] )

order = 3
mesh = Mesh(OCCGeometry(shape).GenerateMesh(maxh=0.5)).Curve(order)
#Draw (mesh);

Consider the mixed problem:

- bottom sphere: Dirichlet boundary with given $\gamma_D \, E = \boldsymbol m$ (PEC)
- top sphere: Neumann boundary with given $\gamma_N \, E = \boldsymbol j$ (computed from PEC solution)
  
The Neumann boundary condition is manufactured as follows. We solve Neumann trace of the scattered electric field and rebuild a perfect electric conductor on the whole boundary. 

In [ ]:
order = 3
fesHDiv = HDivSurface(mesh, order=order, complex=True)
uHDiv,vHDiv = fesHDiv.TnT() 

Define the incoming plane wave and compute the given Dirichlet data $\boldsymbol m$: 

In [ ]:
eps0 = 8.854e-12 
mu0 = 4*pi*1e-7
omega = 1.5e9
kappa = omega*sqrt(eps0*mu0)
print("kappa = ", kappa)

E_inc = CF((1,0,0))*exp( -1j * kappa * z )

n = specialcf.normal(3)
m = GridFunction(fesHDiv) 
m.Set( - Cross(n, E_inc) , definedon=mesh.Boundaries(".*"), dual=True) # Hdiv
#Draw(Norm(m), mesh, draw_vol=False, order=2) ;

In [ ]:
# V, K: use Hdiv as TnT  
intorder = 2 * order + 6
with TaskManager(): 
    V = MaxwellSingleLayerPotentialOperator(fesHDiv, kappa, intorder=intorder, eps=1e-4)
    K = MaxwellDoubleLayerPotentialOperatorNew(fesHDiv, kappa, intorder=intorder, eps=1e-4)

In [ ]:
j = GridFunction(fesHDiv) 
pre = BilinearForm( uHDiv.Trace() * vHDiv.Trace() *ds).Assemble().mat.Inverse(freedofs=fesHDiv.FreeDofs()) 
with TaskManager(): 
    M = BilinearForm( Cross( uHDiv.Trace(), n) * vHDiv.Trace()* ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv>  
    rhs = ( (0.5 * M.mat - K.mat ) * m.vec).Evaluate() 
    CG(mat = V.mat, pre=pre, rhs = rhs, sol=j.vec, tol=1e-8, maxsteps=500, initialize=False, printrates=False)

In [ ]:
mDir = GridFunction(fesHDiv)
mDir.Set( m, definedon=mesh.Boundaries("dirichlet"), dual=True) # given Dirichlet data
Draw (Norm(mDir), mesh, draw_vol=False, order=3);

In [ ]:
jNeu = GridFunction(fesHDiv)
jNeu.Set( j, definedon=mesh.Boundaries("neumann"), dual=True) # given Neumann data
Draw (Norm(jNeu), mesh, draw_vol=False, order=3);

In [ ]:
eps = 1e-6
intorder = 2 * order + 6
with TaskManager():
    M = BilinearForm( Cross( uHDiv.Trace(), n) * vHDiv.Trace()* ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv>  
    fd = ((0.5 * M.mat - K.mat) * mDir.vec - V.mat * jNeu.vec).Evaluate()
    fn = ((0.5 * M.mat - K.mat) * jNeu.vec - V.mat * mDir.vec).Evaluate()

In [ ]:
fesHDivN = HDivSurface(mesh, order=order, dirichlet="neumann") # Dirichlet nodes free dofs
fesHDivD = HDivSurface(mesh, order=order, dirichlet="dirichlet") # Neumann nodes free dofs

lhs = BlockMatrix([[V.mat, K.mat], [K.mat, V.mat]]) 
rhs = BlockVector([fd, fn])
pre = BilinearForm( uHDiv.Trace() * vHDiv.Trace() *ds).Assemble()
preBlock = BlockMatrix ([[pre.mat.Inverse(freedofs=fesHDivN.FreeDofs()), None], [None, pre.mat.Inverse(freedofs=fesHDivD.FreeDofs())]])

sol = GMRes(A=lhs, b=rhs, pre=preBlock, maxsteps=300, tol=1e-8, printrates=False)

In [ ]:
gfu = GridFunction(fesHDiv)
gfu.vec[:] = sol[0]
print ("Neumann error =", sqrt(Integrate(Norm(gfu + jNeu - j)**2, mesh.Boundaries(".*"), BND)))
Draw (Norm(gfu), mesh.Boundaries("dirichlet"), order=3);

In [ ]:
# compare with the expected Neuman trace (numerical solution for PEC on whole boundary) 
gfu.vec[:] = sol[1]
print ("Dirichlet error =", sqrt(Integrate(Norm(gfu +mDir - m)**2, mesh.Boundaries(".*"), BND)))
Draw (Norm(gfu), mesh.Boundaries("neumann"), draw_vol=False, order=3);

References: 

[Boundary Element Methods for Maxwell Transmission Problems in Lipschitz Domains](https://www.semanticscholar.org/paper/Boundary-Element-Methods-for-Maxwell-Transmission-Buffa-Hiptmair/92147cb3aee1a71d0a38544d0f6fdd6d1213299c)